In [1]:
import bambi as bmb
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import arviz as az

In [2]:
# import training data and only fit model on respondents that voted
train = pd.read_csv("BES cleaned.csv")
train = train[(train["TO"]==1) & (train["Vote"].isin(["Labour","Conservatives","Liberal Democrats","Other"]))]
train.shape

(1779, 8)

In [3]:
# define and fit model
cat_model = bmb.Model("Vote ~ 1 + C(SexCat) + C(AgeCat) + C(ONSed) + C(Region) + (1|C(ONSConstID))", train, family="categorical")

In [4]:
VI_model = cat_model.fit(draws=100, 
    tune=1000,
    chains=4, 
    random_seed=42,
    init_kwargs={"target_accept": 0.95})

C:\Users\roddy\anaconda3\envs\MRP_map\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [Intercept, C(SexCat), C(AgeCat), C(ONSed), C(Region), 1|C(ONSConstID)_sigma, 1|C(ONSConstID)_offset]


Output()

Sampling 4 chains for 1_000 tune and 100 draw iterations (4_000 + 400 draws total) took 376 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [5]:
# Predict probability of vote for each party in each stratification cell
ONS_df = pd.read_csv("ONS with turnout prediction.csv")
predict_df = ONS_df[['AgeCat', 'SexCat', 'ONSed', 'Region', 'ONSConstID']].drop_duplicates()

In [6]:
cat_model.predict(VI_model, data=predict_df, sample_new_groups=True)

In [7]:
VI_means = VI_model['posterior']['p'].mean(dim=['chain', 'draw'])
VI_hdi = az.hdi(VI_model, hdi_prob=0.95)
VI_lower = VI_hdi['p'].sel(hdi='lower')
VI_upper = VI_hdi['p'].sel(hdi='higher')

In [8]:
predict_df['Con_Vote'] = VI_means[:,0]
predict_df['Con_U95'] = VI_upper[:,0]
predict_df['Con_L95'] = VI_lower[:,0]

predict_df['Lab_Vote'] = VI_means[:,1]
predict_df['Lab_U95'] = VI_upper[:,1]
predict_df['Lab_L95'] = VI_lower[:,1]

predict_df['LD_Vote'] = VI_means[:,2]
predict_df['LD_U95'] = VI_upper[:,2]
predict_df['LD_L95'] = VI_lower[:,2]

predict_df['Oth_Vote'] = VI_means[:,3]
predict_df['Oth_U95'] = VI_upper[:,3]
predict_df['Oth_L95'] = VI_lower[:,3]

In [9]:
ONS_with_VI = pd.merge(ONS_df, predict_df, on=['AgeCat','SexCat', 'ONSed', 'ONSConstID', 'Region'])

In [10]:
ONS_with_VI.to_csv('VI Preds.csv')